<a href="https://colab.research.google.com/github/MahediKamal/thesis/blob/main/Thesis_car_distense_measurement_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# take_video -> split_frames -> iterate_over_frames -> crop_number_plates -> iterate_over_croped_images -> calculate_text_size -> predict_distance
# step-1         step-2           step-3                 step-4               step-5                          step-6                 step-7
%cd /content/drive/MyDrive/Thesis/yolov5
%pip install -qr requirements.txt  # install

import torch
import utils

import cv2 as cv

/content/drive/MyDrive/Thesis/yolov5
     |████████████████████████████████| 1.6 MB 9.1 MB/s 


In [2]:
from PIL import Image as im
%matplotlib inline
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np
import os
from zmq.constants import NULL

In [3]:
# imports for regrassion
! pip install pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:

# ------------define path and variable
number_pate_detection_model_path = '/content/drive/MyDrive/train_data/best_number_plate.pt'
# frame_path = "/content/drive/MyDrive/mahedikamal01/ Thesis /4.jpg"
is_model_ready = False
model = NULL

regression_model = NULL

In [6]:
# ----------------------
def delete_folder():
  !rm -rf '/content/drive/MyDrive/Thesis/yolov5/runs/detect/exp'

def build_model():
  global model
  model = torch.hub.load('ultralytics/yolov5', 'custom', path=number_pate_detection_model_path, force_reload=True)
  global is_model_ready
  is_model_ready = True


def show_detection():
  !python /content/drive/MyDrive/Thesis/yolov5/detect.py --weights '/content/drive/MyDrive/train_data/best_number_plate.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Thesis/training_demo/images/train/18.jpg'
  # display.Image(filename='/content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/licence_pa.jpg', width=600)
  img = cv.imread("/content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/18.jpg")
  plt.subplot(111), plt.imshow(img),plt.title('output')
  # !rm -rf /content/drive/MyDrive/Thesis/yolov5/runs/detect/exp

In [7]:

def build_regression_model():
  df = pd.read_csv('/content/drive/MyDrive/Thesis/train(distance).csv') # train data
  x = df['pixel']
  y = df['distance']

  poly = PolynomialFeatures(degree = 2)
  X_poly = poly.fit_transform(df[['pixel']]) # [x   x^2   x^3 ...]
  
  global regression_model
  regression_model = LinearRegression()
  regression_model.fit(X_poly, y)

In [36]:
def find_length(posi, posj, np_img):
  stack = []
  stack.append([posi,posj])
  x4 = [1, 0, -1, 0, 2, 0, -2, 0, 3, 0, -3, 0];
  y4 = [0, 1, 0, -1, 0, 2, 0, -2, 0, 3, 0, -3];
  
  minh = posi
  maxh = posi
  while(len(stack) > 0):
    cur = stack.pop()
    x = cur[0]
    y = cur[1]
    if(x > maxh):
      maxh = x
    if(x < minh):
      minh = x
    np_img[x][y] = 150
    # print(stack)
    for i in range(0, 12):
      dx = x + x4[i]
      dy = y + y4[i]
      # print(dx,dy)
      try:
        if(np_img[dx][dy] == 0):
          stack.append([dx,dy]) 
      except:
        print("index out-of bount during bfs")
  
  # print(maxh, minh, maxh-minh)
  return maxh-minh

In [9]:
def find_black_points(np_img, row, tot_clm):
  lst = []
  for i in range(1,tot_clm-1):
    if(np_img[row][i-1] == 255 and np_img[row][i] == 0):
      lst.append(i)
  return lst

In [10]:
def convert_to_01(np_img):
  np_img[np_img < 130] = 0
  np_img[np_img > 130] = 255
  return np_img

In [11]:
def color_white_sorrounding_digits(crop_img_path):
  #---------------------------------color_white_sorrounding_digits---------
  crp_img = cv.imread(crop_img_path)
  gray = cv.cvtColor(crp_img, cv.COLOR_BGR2GRAY)
  # plt.imshow(cv.cvtColor(gray, cv.COLOR_BGR2RGB))

  arr = np.array(gray)
  #arr[arr > 100] = 0
  x = int((arr.shape[0])/2)
  for i in range(0, x):
    for j in range(0, arr.shape[1]-1):
      arr[i][j] = 255

  x_hf = int(x/2)-1
  y_f = int((arr.shape[1]/9)*2)
  y_s = int(arr.shape[1] - int((arr.shape[1]/9)*2))

  for i in range(x_hf, arr.shape[0]):
    for j in range(0, y_f):
      arr[i][j] = 255

  for i in range(x_hf, arr.shape[0]):
    for j in range(y_s, arr.shape[1]-1):
      arr[i][j] = 255
      
  # data = im.fromarray(arr)
  return arr
  # plt.imshow(data)

In [28]:
def calculate_text_size(crop_img_path):
  
  pre_processed_img_arr1 = color_white_sorrounding_digits(crop_img_path)
  pre_processed_img_arr2 = convert_to_01(pre_processed_img_arr1)

  mid = int((pre_processed_img_arr2.shape[0])/2)
  mid = int(mid + (mid/2))

  lst = find_black_points(pre_processed_img_arr2, mid, pre_processed_img_arr2.shape[1]-1)
  # print(lst)

  mx_sz = 0
  for i in range (0, len(lst)):
    sz = find_length(mid,lst[i],pre_processed_img_arr2)
    if sz > mx_sz:
      mx_sz = sz
  print("calculated number plate size : "+ str(mx_sz))
  # ------------------------------------step-7 ()------------
  build_regression_model()
  poly = PolynomialFeatures(degree = 2)
  global regression_model
  
  print("predicted distace: "+ str(regression_model.predict(poly.fit_transform([[mx_sz]]))))
  print("----")
  
  # data = im.fromarray(pre_processed_img_arr2)
  # plt.imshow(data)
  

In [42]:
  def find_number_plate_coordinates(frame_path):
    global is_model_ready
    if(is_model_ready == False):
      build_model()
    global model
    result = model(frame_path)
    crops = result.crop(save=True) 

    # ------------------------------step-5 (iterate_over_croped_images)------------------------------
    directory = '/content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/crops/number_plate'
    # iterate over files in that directory
    try:
      for filename in os.listdir(directory):
          f = os.path.join(directory, filename)
          # checking if it is a file
          if os.path.isfile(f):
            
            # ------------------------------step-6 (calculate_text_size)------------------------------
            print("current cropped number plate path: "+ f)
            text_sise = calculate_text_size(f)
    except:
      print("no number plate found in the frame")

In [33]:
# ------------------------------step-3 (iterate_over_frames)---------------------------------------
# after splitting images we will save the frames inside a folder name 'frames'
def iterate_over_frames():
  # assign directory
  directory = '/content/drive/MyDrive/mahedikamal01/ Thesis /frames'
  
  # iterate over files in
  # that directory
  for filename in os.listdir(directory):
      f = os.path.join(directory, filename)
      # checking if it is a file
      if os.path.isfile(f):
        # ------------------------------step-4 (crop_number_plates)--------------------------------
        print("*********new frame*************")
        print("current frame path: "+ f)
        find_number_plate_coordinates(f) # finds number plate and crot it
        delete_folder()
        # break

In [43]:
iterate_over_frames()

*********new frame*************
current frame path: /content/drive/MyDrive/mahedikamal01/ Thesis /frames/4.8.jpg


INFO:yolov5:Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp
INFO:yolov5:Saved results to runs/detect/exp

Saved results to runs/detect/exp



current cropped number plate path: /content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/crops/number_plate/4.8.jpg
calculated number plate size : 55
predicted distace: [      12.19]
----
current cropped number plate path: /content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/crops/number_plate/4.82.jpg
calculated number plate size : 52
predicted distace: [     12.941]
----
*********new frame*************
current frame path: /content/drive/MyDrive/mahedikamal01/ Thesis /frames/3.2.jpg


INFO:yolov5:Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp
INFO:yolov5:Saved results to runs/detect/exp

Saved results to runs/detect/exp



current cropped number plate path: /content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/crops/number_plate/3.2.jpg
calculated number plate size : 83
predicted distace: [     7.7529]
----
*********new frame*************
current frame path: /content/drive/MyDrive/mahedikamal01/ Thesis /frames/3.6.jpg


...

current cropped number plate path: /content/drive/MyDrive/Thesis/yolov5/runs/detect/exp/crops/number_plate/2.8.jpg
calculated number plate size : 93
predicted distace: [     7.2944]
----
*********new frame*************
current frame path: /content/drive/MyDrive/mahedikamal01/ Thesis /frames/6.4.jpg


INFO:yolov5:Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp
INFO:yolov5:Saved results to runs/detect/exp

Saved results to runs/detect/exp



Streaming output truncated to the last 5000 lines.
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount during bfs
index out-of bount 

INFO:yolov5:Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp
INFO:yolov5:Saved results to runs/detect/exp

Saved results to runs/detect/exp



no number plate found in the frame


In [40]:
delete_folder()